In [18]:
# 필요한 Module import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import random
import os

# sklearn
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
# 사실은 one-hot encopding처리는
# pandas를 이용해서 더 많이 처리합니다.
from sklearn.model_selection import train_test_split
# 이상치 처리를 위해
from scipy import stats

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import TensorBoard

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim


In [19]:
# Seed 설정
def seed_everything(seed=42):

    # 1. 환경변수
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 2. Python, Numpy
    random.seed(seed)
    np.random.seed(seed)

    # 3. TensorFlow
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    tf.random.set_seed(seed)

    # 4. PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)  # PyTorch 1.8+ 권장

    print(f"Seeds set to {seed}")

seed_everything(42)

Seeds set to 42


In [20]:
# 1. Raw Data Loading
df = pd.read_csv('./Kaggle_project/Taitanic/data/train.csv')

df
# 필요없는 컬럼을 찾아서 삭제
# 다행히 우리 데이터는 컬럼명이 존재하고
# Data Dictionary가 있어요. 그래서 조금 더 쉽게 컬럼을 제어할 수
# 있어요!
# 상식에 기반해서 제거할 수 있어요 -> 확실한 방법은 아니예요!
# 실제로는 상관계수같은걸 계산해서 수치적으로 필요없는 컬럼을 찾아서
# 제거하는게 좋아요!
# 필요없는 컬럼(독립변수) 4개로 설정
# PassengerId, Name, Ticket, Cabin

# 그런데..가만히 보니 이름안에 title이 포함되어 있어요!
# 이런 데이터를 뽑아서 새로 Feature로 만들면 좋을 듯 해요!
# 이렇게 새로 Feature를 추출해서 사용하는 방법도 많이 사용하는데
# 지금 같은 경우는 Mr, Mrs, Miss, Master 를 추출해서 사용하면
# Sex와 상당히 유사한 정보가 되요! (강하게 중복되고 거의 완전히
# 상관성이 있다고 여길 수 있어요!)
# => 다중공선성이 생기게 되요. feature redundancy(특성 중복)
# => Multicollinearity
# 다중공선성은
# 1. 모델이 혼란스러워해요!
# 2. 회귀계수가 안정되지 못해요! -> 일반화 성능이 떨어져요!
# 그래서.. 우리는 Name에서 Title을 분리하지 않을꺼예요!

df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],
             axis=1,
             inplace=False)
df

# 결측치와 각 컬럼의 값이 어떻게 구성되어 있는지 확인
df.isnull().sum()
# 결측치가 있는 컬럼은 Age(177), Embarked(2)
df['Sex'].value_counts() # male, female 2개만 존재
df['Embarked'].value_counts() # S, C, Q 3개만 존재

# 데이터 타입부터 먼저 변경해줘요!
# 성별부터 문자를 숫자로 변경
gender_mapping = { 'male': 0, 'female': 1 }
df['Sex'] = df['Sex'].map(gender_mapping)
df

# 가족수부터 처리해요!
df['Family'] = df['SibSp'] + df['Parch']
df = df.drop(['SibSp', 'Parch'], axis=1, inplace=False)
df

# Embarked는 결측치를 2개 가지고 있어요!
# 빈도가 가장 많은 값을 찾아서 그 값으로 먼저 결측치 부터
# 채워줄꺼에요!
Embarked_freq = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(Embarked_freq)

# 문자를 숫자로 변경
embarked_mapping = { 'S': 0, 'C' : 1, 'Q' : 2}
df['Embarked'] = df['Embarked'].map(embarked_mapping)

df = pd.get_dummies(df,
                    columns=['Embarked'],
                    drop_first=True)
df

# Age에 대해 결측치부터 처리 => 고민되요! => 평균으로 할께요
df['Age'] = df['Age'].fillna(df['Age'].mean())
# Age에 대한 Bining처리
# 알고리즘(라이브러리)을 이용한 처리가 권장되요!
df['AgeBand'] = pd.cut(df['Age'],
                       5,
                       labels=[0,1,2,3,4])

# Age에 대한 One-Hot처리
df = pd.get_dummies(df,
                    columns=['AgeBand'],
                    drop_first=True)

# Fare에 대해서도 Bining처리를 진행
df['FareBand'] = pd.cut(df['Fare'],
                       4,
                       labels=[0,1,2,3])
df = df.drop('Fare', axis=1, inplace=False)
# One-Hot으로 변경
df = pd.get_dummies(df,
                    columns=['FareBand'],
                    drop_first=True).astype(int)
df.astype(int)

df.shape # (891, 14)

# 정규화는 필요없어요

# 데이터 분할만 하면 될 듯 해요
x_data = df.drop('Survived', axis=1).values
y_data = df['Survived'].values.reshape(-1,1)

x_data_train, x_data_test, y_data_train, y_data_test = \
train_test_split(x_data,
                 y_data,
                 test_size=0.2,
                 stratify=y_data,
                 random_state=42)

In [21]:
# sklearn Model 구현
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score

sklearn_model = linear_model.LogisticRegression()

# Model을 생성했으니 validation 실행
# K-Fold Cross Validation
score = cross_val_score(estimator=sklearn_model,
                        X=x_data,
                        y=y_data,
                        scoring='accuracy',
                        cv=5)

score.mean() # 0.8035904839620865

# 모델 학습
sklearn_model.fit(x_data_train,
                  y_data_train)

# test데이터를 이용해서 예측값 도출
sklearn_y_pred = sklearn_model.predict(x_data_test)

# 예측값과 정답을 이용해서 실제 모델의 정확도를 측정
sklearn_accuracy = accuracy_score(y_data_test, sklearn_y_pred)
sklearn_f1 = f1_score(y_data_test, sklearn_y_pred)

print(sklearn_accuracy, sklearn_f1)
# sklearn_accuracy : 0.7932960893854749
# sklearn_f1 : 0.7086614173228346

0.7932960893854749 0.7086614173228346


/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kiyong/anaconda3/envs/data_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was ex

In [22]:
# Tensorflow 구현

# 모델생성
keras_model = Sequential()

# 레이어 추가
keras_model.add(Input(shape=(13,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))

# 데이터는 Train / Valid / Test 3개로 구분
# 모델 compile
keras_model.compile(optimizer=Adam(learning_rate=1e-3),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# callback 처리
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tb_cb = TensorBoard(log_dir=log_dir,
                    histogram_freq = 1)

# 모델학습
keras_model.fit(x_data_train,
                y_data_train,
                epochs=200,
                callbacks=[tb_cb],
                batch_size = 64,
                validation_split=0.3,
                verbose=1)

'''
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6024 - loss: 1.1235 - val_accuracy: 0.5935 - val_loss: 1.1067
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5924 - loss: 1.0959 - val_accuracy: 0.5794 - val_loss: 1.0773
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5803 - loss: 1.0726 - val_accuracy: 0.5841 - val_loss: 1.0522
Epoch 13/200
...
Epoch 199/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8012 - loss: 0.4929 - val_accuracy: 0.7804 - val_loss: 0.5095
Epoch 200/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8012 - loss: 0.4925 - val_accuracy: 0.7804 - val_loss: 0.5091
'''

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6245 - loss: 1.6190 - val_accuracy: 0.6075 - val_loss: 1.6316
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6225 - loss: 1.5437 - val_accuracy: 0.6075 - val_loss: 1.5514
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6225 - loss: 1.4723 - val_accuracy: 0.6028 - val_loss: 1.4756
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6205 - loss: 1.4055 - val_accuracy: 0.5935 - val_loss: 1.4047
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6205 - loss: 1.3438 - val_accuracy: 0.5888 - val_loss: 1.3395
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6225 - loss: 1.2879 - val_accuracy: 0.5888 - val_loss: 1.2804
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6265 - loss: 1.2379 - val_accuracy: 0.6075 - val_loss: 1.2277
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6245 - loss: 1.1940 - val_accuracy: 0.5981 - val_loss: 1.1813

'\nEpoch 10/200\n8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6024 - loss: 1.1235 - val_accuracy: 0.5935 - val_loss: 1.1067\nEpoch 11/200\n8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5924 - loss: 1.0959 - val_accuracy: 0.5794 - val_loss: 1.0773\nEpoch 12/200\n8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5803 - loss: 1.0726 - val_accuracy: 0.5841 - val_loss: 1.0522\nEpoch 13/200\n...\nEpoch 199/200\n8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8012 - loss: 0.4929 - val_accuracy: 0.7804 - val_loss: 0.5095\nEpoch 200/200\n8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8012 - loss: 0.4925 - val_accuracy: 0.7804 - val_loss: 0.5091\n'

In [23]:
# 예측작업
keras_y_pred = keras_model.predict(x_data_test)
keras_y_pred_class = (keras_y_pred >= 0.5).astype(int)

keras_accuracy = accuracy_score(y_data_test, keras_y_pred_class)
keras_f1 = f1_score(y_data_test, keras_y_pred_class)

print(keras_accuracy, keras_f1) # 0.776536312849162 0.6491228070175439

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
0.776536312849162 0.6491228070175439


In [24]:
# PyTorch

# 유사하게 구현하면 되요!
# 구현했다고 치고!!

In [25]:
# 이렇게 3개의 모델을 다 만들었으면 가장 성능이 좋은 모델을
# 이용해서 결과 파일을 만들어서 케글에 제출!

# 모델은 sklearn모델을 이용하도록 해요!
# 일단 test데이터와 제출파일을 DataFrame으로 불러와요!
test = pd.read_csv('./Kaggle_project/Taitanic/data/test.csv')
submission = pd.read_csv('./Kaggle_project/Taitanic/data/gender_submission.csv')

# test 데이터로 우리 모델에 입력을 넣어서
# 예측값을 얻을려면 당연히... 전처리 해야 해요!
test

test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],
             axis=1,
             inplace=False)
test

# 결측치와 각 컬럼의 값이 어떻게 구성되어 있는지 확인
# test.isnull().sum()

# 데이터 타입부터 먼저 변경해줘요!
# 성별부터 문자를 숫자로 변경
gender_mapping = { 'male': 0, 'female': 1 }
test['Sex'] = test['Sex'].map(gender_mapping)
test

# 가족수부터 처리해요!
test['Family'] = test['SibSp'] + test['Parch']
test = test.drop(['SibSp', 'Parch'], axis=1, inplace=False)
test

# 문자를 숫자로 변경
embarked_mapping = { 'S': 0, 'C' : 1, 'Q' : 2}
test['Embarked'] = test['Embarked'].map(embarked_mapping)

test = pd.get_dummies(test,
                    columns=['Embarked'],
                    drop_first=True)
test

# Age에 대해 결측치부터 처리 => 고민되요! => 평균으로 할께요
test['Age'] = test['Age'].fillna(test['Age'].mean())
# Age에 대한 Bining처리
# 알고리즘(라이브러리)을 이용한 처리가 권장되요!
test['AgeBand'] = pd.cut(test['Age'],
                       5,
                       labels=[0,1,2,3,4])
test = test.drop('Age', axis=1, inplace=False)
test
# Age에 대한 One-Hot처리
test = pd.get_dummies(test,
                    columns=['AgeBand'],
                    drop_first=True)
test

# Fare에 대해서도 결측치 및 Bining처리를 진행
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
test['FareBand'] = pd.cut(test['Fare'],
                       4,
                       labels=[0,1,2,3])
test = test.drop('Fare', axis=1, inplace=False)
# One-Hot처리
test = pd.get_dummies(test,
                    columns=['FareBand'],
                    drop_first=True)

# Pclass에 대한 one-Hot 처리
test = pd.get_dummies(test,
                    columns=['Pclass'],
                    drop_first=True).astype(int)

test.shape # (418, 13)
test

predict = sklearn_model.predict(test.values)

submission['Survived'] = predict

submission.to_csv('./my_titanic.csv',
                  index=False)
